In [1]:
import pandas as pd
import utils
import plotly.graph_objects as go

In [2]:
def plot_candles(df_plot, df_markers=None):
    fig = go.Figure()

    fig.add_trace(go.Candlestick(
        x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
        line=dict(width=1), opacity=1,
        increasing_fillcolor='#24A06B',
        decreasing_fillcolor="#CC2E3C",
        increasing_line_color='#2EC886',  
        decreasing_line_color='#FF3A4C'
    ))

    if df_markers is not None:
        fig.add_trace(go.Candlestick(
            x=df_markers.time, open=df_markers.mid_o, high=df_markers.mid_h, low=df_markers.mid_l, close=df_markers.mid_c,
            line=dict(width=1), opacity=1,
            increasing_fillcolor='#3480eb',
            decreasing_fillcolor="#3480eb",
            increasing_line_color='#3480eb',  
            decreasing_line_color='#3480eb',
            
        ))
        

    fig.update_layout(width=1000,height=400,
        margin=dict(l=10,r=10,b=10,t=10),
        font=dict(size=10,color="#e1e1e1"),
        paper_bgcolor="#1e1e1e",
        plot_bgcolor="#1e1e1e")
    fig.update_xaxes(
        gridcolor="#1f292f",
        showgrid=True,fixedrange=True,rangeslider=dict(visible=False),
        rangebreaks=[
            dict(bounds=["sat", "mon"])
        ]
    )
    fig.update_yaxes(
        gridcolor="#1f292f",
        showgrid=True
    )
    fig.show()

In [3]:
pair = "USD_JPY"
granularity = "H4"
df_raw = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

In [4]:
cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']

In [5]:
df_raw.head(0)

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c


In [6]:
SMALL_BODY = 0.2
CLOSE_DISTANCE = 0.15
DOJI_BODY = 0.05
FULL_BODY = 0.95
CENTER_DISTANCE_HIGH = 0.55
CENTER_DISTANCE_LOW = 0.45
BIG_BODY = 0.6
ENGULFING_FACTOR = 1.1

# Helper functions
def apply_top_end_distance(row):
    if row.DIRECTION == 1:
        return row.mid_h - row.mid_c
    else:
        return row.mid_h - row.mid_o

def apply_bottom_end_distance(row):
    if row.DIRECTION == 1:
        return row.mid_o - row.mid_l
    else:
        return row.mid_c - row.mid_l

# Pattern detection functions
def apply_hammer(row):
    if row.BODY_PERC < SMALL_BODY:
        if row.DIST_TOP_PERC < CLOSE_DISTANCE or row.DIST_BOTTOM_PERC < CLOSE_DISTANCE:
            return True
    return False

def apply_spinning_top(row):
    if row.BODY_PERC < SMALL_BODY:
        if row.DIST_TOP_PERC < CENTER_DISTANCE_HIGH and row.DIST_TOP_PERC > CENTER_DISTANCE_LOW:
            return True
    return False

def apply_engulfing(row):
    if row.PREV_DIRECTION != row.DIRECTION:
        if row.PREV_BODY_PERC > BIG_BODY and row.BODY_PERC > BIG_BODY:
            if row.BODY_RANGE > row.PREV_BODY_RANGE * ENGULFING_FACTOR:
                return True
    return False

def apply_piercing_line(row):
    if row.PREV_DIRECTION == -1 and row.DIRECTION == 1:
        opens_lower = row.mid_o < row.PREV_mid_c
        closes_more_than_halfway = row.mid_c > (row.PREV_mid_o + 0.5 * (row.PREV_mid_c - row.PREV_mid_o))
        if opens_lower and closes_more_than_halfway:
            return True
    return False

# Function to compute statistics
def apply_stats(df):
    df['RANGE'] = df.mid_h - df.mid_l
    df['BODY_RANGE'] = abs(df.mid_c - df.mid_o)
    df['CENTER'] = (df.mid_h - df.mid_l) / 2 + df.mid_l
    df['BODY_PERC'] = df.BODY_RANGE / df.RANGE
    df['DIRECTION'] = df.mid_c - df.mid_o
    df['DIRECTION'] = df['DIRECTION'].apply(lambda x: 1 if x >= 0 else -1)
    df['DIST_TOP'] = df.apply(apply_top_end_distance, axis=1)
    df['DIST_BOTTOM'] = df.apply(apply_bottom_end_distance, axis=1)
    df['DIST_TOP_PERC'] = df.DIST_TOP / df.RANGE
    df['DIST_BOTTOM_PERC'] = df.DIST_BOTTOM / df.RANGE

    # Compute previous candle statistics
    df['PREV_BODY_RANGE'] = df.BODY_RANGE.shift(1)
    df['PREV_BODY_PERC'] = df.BODY_PERC.shift(1)
    df['PREV_DIRECTION'] = df.DIRECTION.shift(1)
    df['PREV_mid_o'] = df.mid_o.shift(1)
    df['PREV_mid_c'] = df.mid_c.shift(1)
    df['PREV_mid_o2'] = df.mid_o.shift(2)
    df['PREV_mid_c2'] = df.mid_c.shift(2)
    df['PREV_BODY_PERC2'] = df.BODY_PERC.shift(2)
    df['PREV_DIRECTION2'] = df.DIRECTION.shift(2)
    
    df.dropna(inplace=True)

    return df

# Function to apply all patterns
def apply_patterns(df):
    df['HAMMER'] = df.apply(apply_hammer, axis=1)
    df['SPINNING_TOP'] = df.apply(apply_spinning_top, axis=1)
    df['DOJI'] = df['BODY_PERC'].apply(lambda x: True if x < DOJI_BODY else False)
    df['MARUBOZU'] = df['BODY_PERC'].apply(lambda x: True if x > FULL_BODY else False)
    df['ENGULFING'] = df.apply(apply_engulfing, axis=1)
    df['PIERCING_LINE'] = df.apply(apply_piercing_line, axis=1)
    
    return df

In [9]:
df = df_raw[cols].copy()
df = apply_stats(df)
df = apply_patterns(df)

In [10]:
df_plot = df.iloc[740:785]
df_markers = df_plot[(df_plot.PIERCING_LINE==True)]



In [11]:
plot_candles(df_plot, df_markers)